In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import warnings
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans

warnings.filterwarnings('ignore')

In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'heartdisease'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

heartdisease_df = pd.read_sql_query('select * from heartdisease',con=engine)

engine.dispose()

In [3]:
# Define the features and the outcome
X = heartdisease_df.iloc[:, :13]
y = heartdisease_df.iloc[:, 13]

# Replace missing values (marked by ?) with a 0
X = X.replace(to_replace='?', value=0)

# Binarize y so that 1 means heart disease diagnosis and 0 means no diagnosis
y = np.where(y > 0, 0, 1)

In [4]:
heartdisease_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
0     303 non-null float64
1     303 non-null float64
2     303 non-null float64
3     303 non-null float64
4     303 non-null float64
5     303 non-null float64
6     303 non-null float64
7     303 non-null float64
8     303 non-null float64
9     303 non-null float64
10    303 non-null float64
11    303 non-null object
12    303 non-null object
13    303 non-null int64
dtypes: float64(11), int64(1), object(2)
memory usage: 33.2+ KB


In [5]:
heartdisease_df.head(20)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0
5,56.0,1.0,2.0,120.0,236.0,0.0,0.0,178.0,0.0,0.8,1.0,0.0,3.0,0
6,62.0,0.0,4.0,140.0,268.0,0.0,2.0,160.0,0.0,3.6,3.0,2.0,3.0,3
7,57.0,0.0,4.0,120.0,354.0,0.0,0.0,163.0,1.0,0.6,1.0,0.0,3.0,0
8,63.0,1.0,4.0,130.0,254.0,0.0,2.0,147.0,0.0,1.4,2.0,1.0,7.0,2
9,53.0,1.0,4.0,140.0,203.0,1.0,2.0,155.0,1.0,3.1,3.0,0.0,7.0,1


In [6]:
# checking for missing data

heartdisease_df.isnull().sum()*100/heartdisease_df.isnull().count()

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
dtype: float64

In [7]:
scaler = StandardScaler()
scaled_X = scaler.fit_transform(X)

In [8]:
# applying k-means


k_means = KMeans(n_clusters=2, random_state=58)
%timeit k_means.fit(scaled_X)
#y_pred = k_means.predict(scaled_X)
k_means.fit(scaled_X)
print(k_means.labels_)

print(type(k_means.labels_))

18.8 ms ± 953 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
[0 0 0 1 1 1 0 1 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 1 0 1 1 1 1 0
 0 0 0 0 1 1 1 1 0 1 0 1 1 1 1 1 1 0 0 1 1 1 1 0 1 0 1 0 0 1 1 0 1 1 0 0 0
 1 1 0 1 1 0 0 1 1 0 1 1 1 1 1 1 1 0 0 1 1 0 0 0 1 1 1 1 1 1 0 1 0 0 0 1 0
 0 1 0 0 1 1 1 0 0 0 0 1 0 1 1 0 0 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 0 0 1 0 1
 1 1 1 1 0 0 0 0 0 1 0 1 1 0 1 1 1 1 1 1 1 1 0 0 1 1 0 0 0 0 1 1 1 0 1 0 1
 1 1 0 1 0 1 0 0 0 1 0 1 1 1 1 1 1 1 1 1 0 0 0 1 0 1 1 1 0 1 1 1 1 0 1 1 1
 1 0 0 1 1 1 0 0 1 0 1 1 1 0 0 1 1 1 1 1 1 1 1 0 1 0 1 1 0 0 0 1 1 1 1 1 1
 1 1 1 1 1 0 0 1 0 1 1 0 0 0 1 1 1 1 1 1 1 0 1 0 1 1 0 0 1 1 1 0 1 0 0 1 1
 0 0 1 0 0 1 1]
<class 'numpy.ndarray'>


In [9]:
y


array([1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,

In [12]:
from sklearn.metrics import accuracy_score
pred_labels=k_means.labels_
accuracy_score(y, pred_labels)

0.8316831683168316

In [ ]:
pred_labels=k_means.labels_
pred_labels=pred_labels.flatten()
pred_labels=pd.DataFrame(pred_labels)

y=y.flatten()
y=pd.DataFrame(y)


In [ ]:
compare=pd.concat([pred_labels,y])

In [13]:
# try to set n_cluster =3

k_means_3 = KMeans(n_clusters=3, random_state=112)
%timeit k_means_3.fit(scaled_X)
#y_pred = k_means.predict(scaled_X)
k_means_3.fit(scaled_X)
print(k_means_3.labels_)

print(type(k_means_3.labels_))

57.2 ms ± 642 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
[2 2 2 0 1 0 2 1 2 2 0 1 2 0 0 0 0 0 1 0 0 1 0 2 2 1 1 1 0 2 1 2 0 0 0 0 2
 2 2 2 2 0 1 0 1 2 0 2 1 0 1 0 0 0 2 2 0 0 0 0 2 1 2 1 2 2 0 0 2 0 1 2 2 0
 0 1 2 1 0 2 2 1 0 2 0 0 0 1 1 1 1 2 2 1 1 0 2 2 0 0 0 0 1 1 2 0 0 0 2 0 2
 2 0 2 2 0 0 1 2 2 2 2 0 2 0 1 2 2 0 1 0 0 0 0 1 1 2 2 2 0 0 0 0 2 2 0 2 0
 0 1 0 1 1 2 2 2 0 0 2 0 0 2 1 1 0 0 0 1 0 1 2 2 1 1 2 2 0 2 0 0 0 2 0 2 1
 1 0 2 0 2 0 2 2 2 1 2 0 1 1 0 1 1 0 1 0 2 2 2 0 1 1 0 0 2 0 0 1 1 1 0 1 1
 1 2 1 0 0 1 2 2 1 2 0 1 1 2 2 0 1 0 0 1 1 0 1 2 0 2 0 0 2 2 2 1 0 1 1 1 0
 0 1 1 1 0 2 2 0 2 0 0 2 2 2 1 0 0 1 1 0 1 2 0 2 0 0 2 2 0 0 0 2 1 2 2 1 0
 2 2 0 2 2 1 0]
<class 'numpy.ndarray'>


In [14]:
pred_labels_3=k_means_3.labels_
accuracy_score(y, pred_labels_3)

0.35973597359735976